# The Battle of Neighborhoods

## Introduction/Business Problem

Bubble tea is growing more and more popular, bubble tea stores are often have a long queues in some hot business area in Beijing. Asian cuisine is also very popular in the US, especially cities full of Asian immigrants, like the Silicon Valley in California. This project aims to find the best location to open a business in a city in Silicon Valley, San Jose, CA.

It’s important to know if it is profitable prior to launching any business, so this report will try to gather data about similar businesses in the neighborhood and potential customer distribution, competitors, etc.  The data can be used as part of a business plan to support making decision. 

Bubble tea is growing more and more popular, bubble tea stores are often have a long queues in some hot business area in Beijing. Asian cuisine is also very popular in the US, especially cities full of Asian immigrants, like the Silicon Valley in California. This project aims to find the best location to open a business in a city in Silicon Valley, San Jose, CA.

It’s important to know if it is profitable prior to launching any business, so this report will try to gather data about similar businesses in the neighborhood and potential customer distribution, competitors, etc.  The data can be used as part of a business plan to support making decision. 


## Data Description

I will use Foursquare API to find venues:  

For competitor study  

-  Existing Bubble Tea shops around the neighbourhoods in San Jose  
-  Similar businesses around the neighbourhoods in San Jose 

For potential customer distribution study 

-  Universities around the neighbourhoods in San Jose  
-  Companies around the neighbourhoods in San Jose 

Data visualization: 
I will use folium the map rendering library to visualize the venues 


In [1]:
#pip3 install numpy
#pip3 install pandas
#pip3 install seaborn
#pip3 install geopy
#pip3 install matplotlib
#pip3 install folium
#pip3 install pyproj
#pip3 install tqdm



In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle

# Visualization
import matplotlib.pyplot
import seaborn as sns
# See full dataframe...
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import math

In [3]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
#!conda install -c conda-forge pyproj 
import pyproj

        
print('Libraries imported.')

Libraries imported.


In [66]:
CLIENT_ID = 'VLEDCUIZD1CXTQVN0GJPI5D4TCWV3PPW32Y52LWOUPS1CBZK' # My Foursquare ID
CLIENT_SECRET = 'HFONPPRLXX35IYQB2ZJJMTTMR1VCSW2DJ4FAM0MFMTQSECWP' # My Foursquare Secret
VERSION = '20200405'
#LIMIT = 30
print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: VLEDCUIZD1CXTQVN0GJPI5D4TCWV3PPW32Y52LWOUPS1CBZK
CLIENT_SECRET:HFONPPRLXX35IYQB2ZJJMTTMR1VCSW2DJ4FAM0MFMTQSECWP


In [33]:
address = 'San Jose, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
#location = geolocator.geocode(address)
#sj_latitude = location.latitude
#sj_longitude = location.longitude

#san jose
sj_latitude = 37.3361905 
sj_longitude = -121.8905833

#san fransisco
#sj_latitude = 45.3842702 
#sj_longitude = -73.995482

# 45.3842702 -73.995482 for San Francisco


print(sj_latitude, sj_longitude)

37.3361905 -121.8905833


## City partitionning

Create a hexagonal grid of cells: offset every other row, and adjust vertical row spacing so that every cell center is equally distant from all it's neighbors.


In [34]:
#!conda install -c conda-forge pyproj 
#import pyproj

In [35]:
# To calculate distances between venues, create the following function to convert between latitude/longitude and (X/Y coordinates in meters)
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('San Jose center longitude={}, latitude={}'.format(sj_longitude, sj_latitude))
x, y = lonlat_to_xy(sj_longitude, sj_latitude)
print('San Jose center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('San Jose center longitude={}, latitude={}'.format(lo, la))


Coordinate transformation check
-------------------------------
San Jose center longitude=-121.8905833, latitude=37.3361905
San Jose center UTM X=-3387418.5623595486, Y=14868887.803689208
San Jose center longitude=-121.8905833, latitude=37.336190500000015


In [36]:
sj_x, sj_y = lonlat_to_xy(sj_longitude, sj_latitude) # City center in Cartesian coordinates
nb_k = 10
radius = 100
k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = sj_x - radius*10
x_step = radius*2
y_min = sj_y - radius*2 - (int(nb_k/k)*k*radius*2 - radius*10)/2
y_step = radius*2 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(nb_k/k)):
    y = y_min + i * y_step
    x_offset = radius if i%2==0 else 0
    for j in range(0, nb_k):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(sj_x, sj_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)
            
map_sj = folium.Map(location=[sj_latitude, sj_longitude], zoom_start=15)
folium.Marker([sj_latitude, sj_longitude], popup='San Jose').add_to(map_sj)
for lat, lon in zip(latitudes, longitudes):
    folium.Circle([lat, lon], radius=radius, color='blue', fill=False).add_to(map_sj)
map_sj

In [64]:
def get_address(latitude, longitude, verbose=False):
    try:
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, 'match')
        response = requests.get(url).json()
        if verbose:
            print('Foursquare API JSON result =>', response)
        results = response['results']
        address = results[0]['formattedAddress']
        return address
    except:
        return None

addr = get_address(sj_latitude, sj_longitude)
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(sj_latitude, sj_longitude, addr))

Reverse geocoding check
-----------------------
Address of [37.3361905, -121.8905833] is: None


In [38]:
addresses = []
count = 0

df_locations = pd.DataFrame()
loaded = False
try:
    with open('locations.pkl', 'rb') as f:
        df_locations = pickle.load(f)
    print('Location data loaded from pickle.')
#    loaded = True
except:
    pass


if not loaded:
    print('Obtaining location addresses: ', end='')
    for lat, lon in zip(latitudes, longitudes):
        count = count + 1
        address = get_address(lat, lon)
        if address is None:
            address = 'NO ADDRESS'
        address = address.replace(', United States', '') # We don't need country part of address
        addresses.append(address)
        if count > 500:
            print("Urgency exit")
            break
    #     print(count)
        print(' .', end='')
    print(' done.')

Location data loaded from pickle.
Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [39]:
if not loaded:
    addresses

In [40]:
if not loaded:
    df_locations = pd.DataFrame({'Address': addresses,
                                 'Latitude': latitudes,
                                 'Longitude': longitudes,
                                 'X': xs,
                                 'Y': ys,
                                 'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,NO ADDRESS,37.337115,-121.880117,-3.388319e+06,1.486824e+07,1111.720843
1,NO ADDRESS,37.337863,-121.881763,-3.388119e+06,1.486824e+07,957.038784
2,NO ADDRESS,37.338611,-121.883410,-3.387919e+06,1.486824e+07,822.145506
3,NO ADDRESS,37.339359,-121.885056,-3.387719e+06,1.486824e+07,718.277964
4,NO ADDRESS,37.340107,-121.886703,-3.387519e+06,1.486824e+07,660.244828
5,NO ADDRESS,37.340855,-121.888350,-3.387319e+06,1.486824e+07,660.244828
6,NO ADDRESS,37.341603,-121.889997,-3.387119e+06,1.486824e+07,718.277964
7,NO ADDRESS,37.342351,-121.891644,-3.386919e+06,1.486824e+07,822.145506
8,NO ADDRESS,37.343099,-121.893291,-3.386719e+06,1.486824e+07,957.038784
9,NO ADDRESS,37.343847,-121.894938,-3.386519e+06,1.486824e+07,1111.720843


In [41]:
df_locations.shape

(110, 6)

In [42]:
df_locations.to_pickle('./locations.pkl')    

In [65]:
# Category IDs are from Foursquare web site 
# https://developer.foursquare.com/docs/resources/categories:

bubble_tea_cat = '52e81612bcbc57f1066b7a0c' # 'bubble tea shop' category venues

# Other categories that might be compatitors: Café, Coffee Shop
competitor_cat = ['4bf58dd8d48988d16d941735','4bf58dd8d48988d1e0931735']

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=350, limit=100):
    version = '20200405'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)  
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [67]:
#test

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, sj_latitude, sj_longitude, competitor_cat, radius=350, limit=100)  
   
results = requests.get(url).json()
results

IndexError: Replacement index 6 out of range for positional args tuple

In [54]:
def get_tea_shops(lats, lons):
    from tqdm.autonotebook import tqdm
    tqdm.pandas()
    bubble_tea_shops = {}
    other_competitors = {}
    location_bubble_teas = []

    print('Obtaining venues around candidate locations:', end='')
    pbar = tqdm(total=len(lats))
    for lat, lon in zip(lats, lons):
        # Using radius=350 to make sure we have overlaps/full coverage so we don't miss bubble tea shops (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, bubble_tea_cat, CLIENT_ID, 
                                          CLIENT_SECRET, radius=1000, limit=100)
        venues1 = get_venues_near_location(lat, lon, competitor_cat, CLIENT_ID, 
                                          CLIENT_SECRET, radius=1000, limit=100)        
#         with open('1. venues.txt', 'w') as outfile:
#             json.dump(venues, outfile)
        area_bubble_tea_shops = []
        area_competitors = []
        print(venues)
        print(venues1)
        
        for venue in venues:            
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
            bubble_tea_shop = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, 
                              venue_distance, venue_categories, x, y)
            if venue_distance<=200:
                    area_bubble_tea_shops.append(bubble_tea_shop)
            bubble_tea_shops[venue_id] = bubble_tea_shop
                
        for venue in venues1:            
#             with open('2. venue.txt', 'w') as outfile:
#                 json.dump(venue, outfile)
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
            other_competitor = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, 
                              venue_distance, venue_categories, x, y)
            other_competitors[venue_id] = other_competitor 
        pbar.update(1)        
        
        location_bubble_teas.append(area_bubble_tea_shops)
#         print(' .', end='')
    pbar.close()
#     print(' done.')
    return bubble_tea_shops, other_competitors, location_bubble_teas

# Try to load from local file system in case we did this before
bubble_tea_shops = {}
other_competitors = {}
location_bubble_teas = []
loaded = False
try:
    with open('bubble_tea_shops_350.pkl', 'rb') as f:
        bubble_tea_shops = pickle.load(f)
    with open('other_competitors_350.pkl', 'rb') as f:
        other_competitors = pickle.load(f)
    with open('location_bubble_teas_350.pkl', 'rb') as f:
        location_bubble_teas = pickle.load(f)
    print('Bubble Tea shop data loaded.')
#loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    bubble_tea_shops, other_competitors, location_bubble_teas = get_tea_shops(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('bubble_tea_shops_350.pkl', 'wb') as f:
        pickle.dump(bubble_tea_shops, f)
    with open('other_competitors_350.pkl', 'wb') as f:
        pickle.dump(other_competitors, f)
    with open('location_bubble_teas_350.pkl', 'wb') as f:
        pickle.dump(location_bubble_teas, f)
        


Bubble Tea shop data loaded.
Obtaining venues around candidate locations:

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]



In [45]:
print('Total number of bubble tea shops:', len(bubble_tea_shops))
print('Total number of other coffee shops or cafes:', len(other_competitors))
print('Percentage of bubble tea shops in the tea/coffee biz: {:.2f}%'.format(len(bubble_tea_shops) / (len(bubble_tea_shops)+len(other_competitors)) * 100))
print('Average number of bubble tea shops in neighborhood:', np.array([len(r) for r in location_bubble_teas]).mean())

Total number of bubble tea shops: 0
Total number of other coffee shops or cafes: 0


ZeroDivisionError: division by zero

In [ ]:
# Visualize existing biz

map_sj = folium.Map(location=[sj_latitude, sj_longitude], zoom_start=15)
folium.Marker([sj_latitude, sj_longitude], popup='Prefecture').add_to(map_sj)
for shop in bubble_tea_shops.values():
    lat = shop[2]; lon = shop[3]
    color = 'red' 
    label = '{}'.format(shop[1])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color,                         
                        popup=label, fill_opacity=1, parse_html=False).add_to(map_sj)
map_sj

## Find all universities and visualize

In [ ]:
# 'Root' category for all universities venues
univ_category = ['4d4b7105d754a06372d81259']

def get_meta_venues(lats, lons, meta_category):
    from tqdm.autonotebook import tqdm
    tqdm.pandas()
    meta_venues = {}
    neighborhoods_venues = []

#     print('Obtaining venues around candidate locations:', end='')
    pbar = tqdm(total=len(lats), desc = 'Obtaining venues', unit= ' coord')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        area_meta_venues = []
        for i, category in enumerate(meta_category):
            venues = get_venues_near_location(lat, lon, category, CLIENT_ID, 
                                              CLIENT_SECRET, radius=350, limit=100)
            for venue in venues:
                venue_id = venue[0]
                venue_name = venue[1]
                venue_categories = venue[2]
                venue_latlon = venue[3]
                venue_address = venue[4]
                venue_distance = venue[5]

                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                bubble_tea_shop = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, 
                                   venue_distance, venue_categories, x, y)
                if venue_distance<=100:
                       area_meta_venues.append(bubble_tea_shop)
                meta_venues[venue_id] = bubble_tea_shop

            neighborhoods_venues.append(area_meta_venues)
#         print(' .', end='')
        pbar.update(1)
    pbar.close()
#     print(' done.')
    return meta_venues, neighborhoods_venues



# plantage = plantage # Just to make sure we won't go after this point

# Try to load from local file system in case we did this before
meta_univ = {}
neighborhoods_univ = []
loaded = False
try:
    with open('meta_univ_350.pkl', 'rb') as f:
        meta_univ = pickle.load(f)
    with open('neighborhoods_univ_350.pkl', 'rb') as f:
        neighborhoods_univ = pickle.load(f)
    print('Universities data loaded.')
#    loaded = True
except:
    pass

if not loaded:
    meta_univ, neighborhoods_univ = get_meta_venues(latitudes, longitudes, univ_category)
    
    # Let's persists this in local file system
    with open('meta_univ_350.pkl', 'wb') as f:
        pickle.dump(meta_univ, f)
    with open('neighborhoods_univ_350.pkl', 'wb') as f:
        pickle.dump(neighborhoods_univ, f)
        
print('Total number of universities:', len(meta_univ))
print('Average number of universities in neighborhood:', np.array([len(r) for r in neighborhoods_univ]).mean())

map_sj = folium.Map(location=[sj_latitude, sj_longitude], zoom_start=15)
folium.Marker([sj_latitude, sj_longitude], popup='Prefecture').add_to(map_sj)
for univ in meta_univ.values():
    lat = univ[2]; lon = univ[3]
    color = 'blue' 
    label = '{}'.format(univ[1])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color,                         
                        popup=label, fill_opacity=1, parse_html=False).add_to(map_sj)
map_sj



## Find all companies and visualize

In [ ]:
# 'Root' category for all companies venues
companies_category = ['4d4b7105d754a06375d81259', '4d4b7105d754a06378d81259', '4d4b7105d754a06379d81259',
                     '4d4b7104d754a06370d81259']

# Try to load from local file system in case we did this before
meta_company = {}
neighborhoods_company = []
loaded = False
try:
    with open('meta_company_350.pkl', 'rb') as f:
        meta_company = pickle.load(f)
    with open('neighborhoods_company_350.pkl', 'rb') as f:
        neighborhoods_company = pickle.load(f)
    print('Companies data loaded.')
#    loaded = True
except:
    pass

if not loaded:
    meta_company, neighborhoods_company = get_meta_venues(latitudes, longitudes, companies_category)
    
    # Let's persists this in local file system
    with open('meta_company_350.pkl', 'wb') as f:
        pickle.dump(meta_company, f)
    with open('neighborhoods_company_350.pkl', 'wb') as f:
        pickle.dump(neighborhoods_company, f)
        
print('Total number of companies:', len(meta_company))
print('Average number of companies in neighborhood:', np.array([len(r) for r in neighborhoods_company]).mean())

map_sj = folium.Map(location=[sj_latitude, sj_longitude], zoom_start=15)
folium.Marker([sj_latitude, sj_longitude], popup='Prefecture').add_to(map_sj)
for comp in meta_company.values():
    lat = comp[2]; lon = comp[3]
    color = 'red'
    label = '{}'.format(comp[1])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color,                         
                        popup=label, fill_opacity=1, parse_html=False).add_to(map_sj)
map_sj



In [ ]:
#visualize distribution of universities, companies and bubble tea shops

map_sj = folium.Map(location=[sj_latitude, sj_longitude], zoom_start=15)
folium.Marker([sj_latitude, sj_longitude], popup='Prefecture').add_to(map_sj)
for comp in meta_company.values():
    lat = comp[2]; lon = comp[3]
    color = 'green'
    label = '{}'.format(comp[1])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color,                         
                        popup=label, fill_opacity=1, parse_html=False).add_to(map_sj)
for univ in meta_univ.values():
    lat = univ[2]; lon = univ[3]
    color = 'yellow'
    label = '{}'.format(univ[1])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color,                         
                        popup=label, fill_opacity=1, parse_html=False).add_to(map_sj)
for shop in bubble_tea_shops.values():
    lat = shop[2]; lon = shop[3]
    color = 'red' 
    label = '{}'.format(shop[1])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color,                         
                        popup=label, fill_opacity=1, parse_html=False).add_to(map_sj)
map_sj

In [ ]:
# Category for all companies + university venues
customer_category = ['4d4b7105d754a06375d81259', '4d4b7105d754a06378d81259', '4d4b7105d754a06379d81259',
                     '4d4b7104d754a06370d81259', '4d4b7105d754a06372d81259']

# Try to load from local file system in case we did this before
meta_customers = []
neighborhoods_customers = []
loaded = False
try:
    with open('meta_customers_350.pkl', 'rb') as f:
        meta_customers = pickle.load(f)
    with open('neighborhoods_customers_350.pkl', 'rb') as f:
        neighborhoods_customers = pickle.load(f)
    print('Customer data loaded.')
#    loaded = True
except:
    pass

if not loaded:
    from tqdm.autonotebook import tqdm
    pbar1 = tqdm(total=len(customer_category), desc = 'Cycling categories', unit= ' categories')
    for category in customer_category:
        meta_customer, neighborhoods_customer = get_meta_venues(latitudes, longitudes, [category])
        meta_customers.append(meta_customer)
        neighborhoods_customers.append(neighborhoods_customer)
        pbar1.update(1)
    pbar1.close()
    
    # Let's persists this in local file system
    with open('meta_customers_350.pkl', 'wb') as f:
        pickle.dump(meta_customers, f)
    with open('neighborhoods_customers_350.pkl', 'wb') as f:
        pickle.dump(neighborhoods_customers, f)
        
print('Total number of customers:', len(meta_customers))
print('Average number of customers in neighborhood:', np.array([len(r) for r in neighborhoods_customers]).mean())

map_sj = folium.Map(location=[sj_latitude, sj_longitude], zoom_start=15)
folium.Marker([sj_latitude, sj_longitude], popup='Prefecture').add_to(map_sj)
for meta_customer in meta_customers:
    for cust in meta_customer.values():
        lat = cust[2]; lon = cust[3]
        color = 'red'
        label = '{}'.format(cust[1])
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color,                         
                            popup=label, fill_opacity=1, parse_html=False).add_to(map_sj)
map_sj


# Analysis

1. Count the number of business in every area candidate:

In [ ]:
counts = []
for i, neighborhoods_customer in enumerate(neighborhoods_customers):
    counts.append([len(cust) for cust in neighborhoods_customers[i]])
final_count= []
len(counts[0])
for m in range(len(counts[0])):
    final_count.append(0)
    for n in range(len(counts)):
        final_count[m] = final_count[m] + counts[n][m]
len(final_count)

In [ ]:
# location_customers_count = [len(cust) for cust in neighborhoods_customers]
counts = []
for i, neighborhoods_customer in enumerate(neighborhoods_customers):
    counts.append([len(cust) for cust in neighborhoods_customers[i]])
location_customers_count= []
len(counts[0])
for m in range(len(counts[0])):
    location_customers_count.append(0)
    for n in range(len(counts)):
        location_customers_count[m] = location_customers_count[m] + counts[n][m]

df_locations['Customers in area'] = location_customers_count

print('Average number of customers in every area with radius=100m:', np.array(location_customers_count).mean())

df_locations.head(5)

In [ ]:
#10 first customer dense areas
df_locations.sort_values(by='Customers in area', ascending=False).head(10)


In [68]:
search_query = 'Bubble Tea'
radius = 3000
print(search_query + ' .... OK!')

Bubble Tea .... OK!


In [70]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, sj_latitude, sj_longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=VLEDCUIZD1CXTQVN0GJPI5D4TCWV3PPW32Y52LWOUPS1CBZK&client_secret=HFONPPRLXX35IYQB2ZJJMTTMR1VCSW2DJ4FAM0MFMTQSECWP&ll=37.3361905,-121.8905833&v=20200405&query=Bubble Tea&radius=3000&limit=30'

In [71]:
results = requests.get(url).json()
results

{'meta': {'code': 429,
  'errorType': 'quota_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '5e8b71b8963d29001b0379e2'},
 'response': {}}

In [ ]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
df = json_normalize(venues)
#df.head()
df.shape

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
df_filtered = df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_filtered['categories'] = df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]

df_count = df_filtered.shape[0]

for i in range(0,df_count-1):
        print('latitude: ', df_filtered.lat[i], 'longitude: ', df_filtered.lng[i], 'category: ', df_filtered.categories[i])
        


In [ ]:
df_filtered.name
#df_filtered.name[1]


In [ ]:
venues_map = folium.Map(location=[sj_latitude, sj_longitude], zoom_start=13) # generate map centred around San Jose

folium.features.CircleMarker(
        [df_filtered.lat[1], df_filtered.lng[1]],
        radius=5,
        color='blue',
        popup=df_filtered.categories[1],
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)




# add a red circle marker to represent San Jose
folium.features.CircleMarker(
    [sj_latitude, sj_longitude],
    radius=10,
    color='red',
    popup='San Jose',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Bubble Tea Shop as blue circle markers
#for lat, lng, label in zip(df_filtered.lat, df_filtered.lng, df_filtered.categories):
for i in range(0,df_count-1):
    folium.features.CircleMarker(
        [df_filtered.lat[i], df_filtered.lng[i]],
        radius=5,
        color='blue',
        popup=df_filtered.categories[i],
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [57]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

NameError: name 'latitude' is not defined

In [ ]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [ ]:
trending_venues_df